In [ ]:
# bunch of the initialization code #

# 1. Optimization

## Kalman Filter

In [ ]:
search_space = [ # 'name' is used directly as a kwarg
    Real(1e-5, 0.1, name='delta', prior='log-uniform'),
    Real(0.5, 4, name='obs_cov_reg', prior='log-uniform'),
    Real(0.001, 0.1, name='trans_cov_avg', prior='log-uniform'),
    Real(0.1, 10, name='obs_cov_avg', prior='log-uniform')
]
SEED = 3178749

# call func
res = bayesian_optimize_workflow(
    execute_workflow_fn=execute_kalman_workflow,
    top_pair_count=10,
    start_year=2008,
    min_end_year=2016,
    max_end_year=2021,
    search_space=search_space,
    n_calls=30,
    seed=SEED,
    verbose=True
)
param_names = [dim.name for dim in search_space]
best_params = {k: res.x[i] for i, k in enumerate(param_names)}
best_mean_mse = res.fun

## Transformer

In [ ]:
search_space = [ # 'name' is used directly as a kwarg
    ...
]
SEED = 3178749

# call func
res = bayesian_optimize_workflow(
    execute_workflow_fn=execute_transformer_workflow,
    top_pair_count=10,
    start_year=2008,
    min_end_year=2016,
    max_end_year=2021,
    search_space=search_space,
    n_calls=30,
    seed=SEED,
    verbose=True
)
param_names = [dim.name for dim in search_space]
best_params = {k: res.x[i] for i, k in enumerate(param_names)}
best_mean_mse = res.fun

## Time-MoE

In [ ]:
search_space = [ # 'name' is used directly as a kwarg
    ...
]
SEED = 3178749

# call func
res = bayesian_optimize_workflow(
    execute_workflow_fn=execute_timemoe_workflow,
    top_pair_count=10,
    start_year=2008,
    min_end_year=2016,
    max_end_year=2021,
    search_space=search_space,
    n_calls=30,
    seed=SEED,
    verbose=True
)
param_names = [dim.name for dim in search_space]
best_params = {k: res.x[i] for i, k in enumerate(param_names)}
best_mean_mse = res.fun

# 2. Results

## Kalman Filter

In [ ]:
# Hard code hyperparameters based on results above
delta: float =
obs_cov_reg: float =
trans_cov_avg: float =
obs_cov_avg: float =

## Transformer

In [ ]:
# Hard code hyperparameters based on results above
## optimized hyperparams: architecture ##
d_model: int =
nhead: int =
num_layers: int =
dropout: float =
## optimized hyperparams: architecture ##
## optimized hyperparams: learning algorithm ##
learning_rate: float =
min_learning_rate: float =
warmup_ratio: float =
weight_decay: float =
batch_size: int =
adam_beta1: float =
adam_beta2: float =
adam_epsilon: float =
## optimized hyperparams: learning algorithm ##

## Time-MoE

In [ ]:
# Hard code hyperparameters based on results above
## optimized hyperparams: learning algorithm ##
learning_rate: float =
min_learning_rate: float =
warmup_ratio: float =
weight_decay: float =
batch_size: int =
adam_beta1: float =
adam_beta2: float =
adam_epsilon: float =
## optimized hyperparams: learning algorithm ##